# Demo 2 — Hyperparameter Tuning with Optuna

Use Optuna to search for the best ARIMA `(p, d, q)` parameters.
Each trial is logged as a **nested MLflow run** — great for comparing results in the UI.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
import optuna
import mlflow
import mlflow.statsmodels
import warnings
warnings.filterwarnings("ignore")

In [3]:
mlflow.set_tracking_uri("http://localhost:5050")
mlflow.set_experiment("temperature-forecast-simple")

<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1771879410391, experiment_id='3', last_update_time=1771879410391, lifecycle_stage='active', name='temperature-forecast-simple', tags={}, workspace='default'>

## Load data & train/test split

In [4]:
df = pd.read_csv("../data/jena_daily_temp.csv", parse_dates=["Date Time"], index_col="Date Time")

train = df.iloc[:-90]
test = df.iloc[-90:]

print(f"Train: {len(train)} days | Test: {len(test)} days")

Train: 2831 days | Test: 90 days


## Optuna objective with nested MLflow runs

In [5]:
def objective(trial):
    p = trial.suggest_int("p", 0, 8)
    d = trial.suggest_int("d", 0, 2)
    q = trial.suggest_int("q", 0, 8)
    order = (p, d, q)

    with mlflow.start_run(run_name=f"arima-{p}-{d}-{q}", nested=True):
        try:
            model = ARIMA(train["temperature"], order=order)
            results = model.fit()

            forecast = results.forecast(steps=len(test))
            rmse = np.sqrt(mean_squared_error(test["temperature"], forecast))

            mlflow.log_params({"p": p, "d": d, "q": q, "order": str(order)})
            mlflow.log_metrics({"rmse": rmse, "aic": results.aic})

            return rmse
        except Exception:
            return float("inf")

## Run the study (~50 trials)

In [6]:
with mlflow.start_run(run_name="optuna-tuning"):
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=50, show_progress_bar=True)

    # Log the best result to the parent run
    best = study.best_trial
    mlflow.log_params({
        "best_p": best.params["p"],
        "best_d": best.params["d"],
        "best_q": best.params["q"],
    })
    mlflow.log_metric("best_rmse", best.value)

print(f"Best order: ({best.params['p']}, {best.params['d']}, {best.params['q']})")
print(f"Best RMSE: {best.value:.2f}")

[I 2026-02-23 21:51:05,441] A new study created in memory with name: no-name-ac4c51ac-f1df-4704-bae3-e1502c82d818


  0%|          | 0/50 [00:00<?, ?it/s]

🏃 View run arima-2-2-4 at: http://localhost:5050/#/experiments/3/runs/1d53ca3b14ee4d009b496e9ef1e25ec1
🧪 View experiment at: http://localhost:5050/#/experiments/3
[I 2026-02-23 21:51:07,463] Trial 0 finished with value: 9.772958974482032 and parameters: {'p': 2, 'd': 2, 'q': 4}. Best is trial 0 with value: 9.772958974482032.
🏃 View run arima-5-2-1 at: http://localhost:5050/#/experiments/3/runs/08257b2a123b494884071eeb69b4468f
🧪 View experiment at: http://localhost:5050/#/experiments/3
[I 2026-02-23 21:51:08,627] Trial 1 finished with value: 11.046161072484805 and parameters: {'p': 5, 'd': 2, 'q': 1}. Best is trial 0 with value: 9.772958974482032.
🏃 View run arima-8-2-5 at: http://localhost:5050/#/experiments/3/runs/0fad2919ae224454bf39f434594283c2
🧪 View experiment at: http://localhost:5050/#/experiments/3
[I 2026-02-23 21:51:12,404] Trial 2 finished with value: 10.970573033361907 and parameters: {'p': 8, 'd': 2, 'q': 5}. Best is trial 0 with value: 9.772958974482032.
🏃 View run arima-

## Bonus: MLflowCallback

Optuna also provides a built-in `MLflowCallback` that logs trials automatically.

In [9]:
from optuna.integration.mlflow import MLflowCallback

mlflow_callback = MLflowCallback(
    tracking_uri="http://localhost:5050",
    metric_name="rmse",
    create_experiment=False,
    mlflow_kwargs={"experiment_id": mlflow.get_experiment_by_name("temperature-forecast-simple").experiment_id},
)


def objective_simple(trial):
    p = trial.suggest_int("p", 0, 8)
    d = trial.suggest_int("d", 0, 2)
    q = trial.suggest_int("q", 0, 8)

    try:
        model = ARIMA(train["temperature"], order=(p, d, q))
        results = model.fit()
        forecast = results.forecast(steps=len(test))
        return np.sqrt(mean_squared_error(test["temperature"], forecast))
    except Exception:
        return float("inf")


study2 = optuna.create_study(direction="minimize")
study2.optimize(objective_simple, n_trials=10, callbacks=[mlflow_callback])

print(f"Best RMSE (callback approach): {study2.best_value:.2f}")

[I 2026-02-23 21:53:19,430] A new study created in memory with name: no-name-626c515b-bfda-439b-805e-e255e1fcbe14
[I 2026-02-23 21:53:20,917] Trial 0 finished with value: 11.551094808824567 and parameters: {'p': 4, 'd': 1, 'q': 6}. Best is trial 0 with value: 11.551094808824567.


🏃 View run 0 at: http://localhost:5050/#/experiments/3/runs/5b1fcbb8f82445db8e26982d2bda7206
🧪 View experiment at: http://localhost:5050/#/experiments/3


[I 2026-02-23 21:53:21,305] Trial 1 finished with value: 24.372229333470425 and parameters: {'p': 4, 'd': 2, 'q': 0}. Best is trial 0 with value: 11.551094808824567.


🏃 View run 1 at: http://localhost:5050/#/experiments/3/runs/20e44610e48448d2bfafd82aaee6eb5b
🧪 View experiment at: http://localhost:5050/#/experiments/3


[I 2026-02-23 21:53:24,929] Trial 2 finished with value: 10.589679007913649 and parameters: {'p': 8, 'd': 1, 'q': 4}. Best is trial 2 with value: 10.589679007913649.


...

🏃 View run 7 at: http://localhost:5050/#/experiments/3/runs/884fb67451b04b47b94a84f0a62a9a36
🧪 View experiment at: http://localhost:5050/#/experiments/3


[I 2026-02-23 21:53:33,931] Trial 8 finished with value: 11.492317288895729 and parameters: {'p': 1, 'd': 1, 'q': 5}. Best is trial 4 with value: 6.939320127368977.


🏃 View run 8 at: http://localhost:5050/#/experiments/3/runs/116c174b5da74a569d7b2d545db2d295
🧪 View experiment at: http://localhost:5050/#/experiments/3


[I 2026-02-23 21:53:38,797] Trial 9 finished with value: 11.965099349857145 and parameters: {'p': 6, 'd': 2, 'q': 8}. Best is trial 4 with value: 6.939320127368977.


🏃 View run 9 at: http://localhost:5050/#/experiments/3/runs/87f427b610e44a4191c9134bd38f1188
🧪 View experiment at: http://localhost:5050/#/experiments/3
Best RMSE (callback approach): 6.94


---
**Next:** Open the MLflow UI at http://localhost:5050, expand the parent run to see nested trials, and sort by RMSE.